In [ ]:
!pip install yfinance
import pandas as pd 
import yfinance as yf # import libraries
import numpy as np
import matplotlib.pyplot as plt
plt.style.use('seaborn')

     |████████████████████████████████| 5.5MB 6.3MB/s 
  Created wheel for yfinance: filename=yfinance-0.1.59-py2.py3-none-any.whl size=23442 sha256=b231eaf21aa0f7ad75e7b83a568843494f235bc80baaae022e9137a80f91d159
  Stored in directory: /root/.cache/pip/wheels/f8/2a/0f/4b5a86e1d52e451757eb6bc17fd899629f0925c777741b6d04
Successfully built yfinance
  Found existing installation: lxml 4.2.6
    Uninstalling lxml-4.2.6:
      Successfully uninstalled lxml-4.2.6


In [ ]:
data = yf.download('ALKEM.NS', 
                      start='2021-04-08', 
                      end='2021-04-13',
                      progress=False)
#data.dropna(inplace=True)
data

,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2021-04-08,2745.000000,2760.0,2704.0,2706.449951,2706.449951,90791
2021-04-09,2710.000000,2882.0,2710.0,2832.300049,2832.300049,630670
2021-04-12,2829.899902,2889.0,2721.0,2745.550049,2745.550049,449475


In [ ]:
value=[]
for i in zip(data['Open'],data['Close']):
  if i[0]>i[1]:
    value.append('Red')
  else:
    value.append('Green')
data['color']= value

In [ ]:
sash=[]
for i in range(0,len(data)):
  try:
    if (data['Close'][i] < data['Open'][i+1]) and (data['Open'][i] > data['Open'][i+1]) and (data['Open'][i] < data['Close'][i+1]) and (data['color'][i] == 'Red') and (data['color'][i+1] == 'Green'):
      sash.append('Bull Sash')
    else:
      sash.append('flat')
  except:
    sash.append('flat')
data['Sash']= sash

In [ ]:
data['Sash']=data['Sash'].shift(1)

In [ ]:
data

,Open,High,Low,Close,Adj Close,Volume,color,Sash
Date,,,,,,,,
2021-04-08,2745.000000,2760.0,2704.0,2706.449951,2706.449951,90791,Red,NaN
2021-04-09,2710.000000,2882.0,2710.0,2832.300049,2832.300049,630670,Green,Bull Sash
2021-04-12,2829.899902,2889.0,2721.0,2745.550049,2745.550049,449475,Red,flat


In [ ]:
data2 = yf.download('ALKEM.NS', 
                       start='2021-04-08', 
                       end='2021-04-13',
                       progress=False, interval= "15m")
# data2.dropna(inplace=True)

In [ ]:
data2.reset_index(inplace=True)

In [ ]:
data2['Datetime']=data2['Datetime'].astype('str')
df=data2['Datetime'].str.split(' ',expand=True)
data2['Date']=df[0]
data2['Time']=df[1]

In [ ]:
##
# NSE WAS CLOSE DUE TO GLICTH

for i in data2['Date']:
  if i=='2021-02-24':
    data2.drop(data2[data2['Date'] == i].index, inplace = True) 

data2.reset_index(inplace=True)
data2.drop('index',axis=1,inplace=True)

data=data.reset_index()

data['Date']=data['Date'].astype('str')

for i in data['Date']:
  if i=='2021-02-24':
    data.drop(data[data['Date'] == i].index, inplace = True) 

data=data.reset_index()
data.drop('index',axis=1,inplace=True)

In [ ]:
##

In [ ]:
data3=data2.copy()

In [ ]:
for i in data2['Time']:
  if i=='10:30:00+05:30' or i=='10:45:00+05:30' or i=='15:00:00+05:30' or i=='15:15:00+05:30':
    pass
  else:
    data2.drop(data2[data2['Time'] == i].index, inplace = True) 

In [ ]:
data2.reset_index(inplace=True)

In [ ]:
data2.drop('index',axis=1,inplace=True)

In [ ]:
#data2
avg_cost=[]
for i in range(0,len(data2)-1):
  avg_cost.append(((data2['High'][i] + data2['Low'][i] + data2['High'][i+1] + data2['Low'][i+1])/4))
avg_cost.append(data2['Open'][i])

In [ ]:
data2['avg buy']=avg_cost

In [ ]:
for i in data3['Time']:
  if i=='10:30:00+05:30'  or i=='09:15:00+05:30' or i=='09:30:00+05:30' or i=='09:45:00+05:30' or i=='10:00:00+05:30' or i=='10:15:00+05:30':
    pass
  else:
    data3.drop(data3[data3['Time'] == i].index, inplace = True) 

In [ ]:
data3=data3.reset_index()

In [ ]:
low=[]
for i in list(data['Date']):
  temp=data3[data3['Date'] == i] 
  low.append(min(temp['Low']))

In [ ]:
data['lowest']=low

In [ ]:
data2=data2.reset_index().drop('index',axis=1)

In [ ]:
data2=data2.drop(['High','Low','Close','Adj Close','Volume','Date','Time'],axis=1)

In [ ]:
data2=data2.rename(columns={'Open':'Buy Price','Datetime':'Buy Date'})

In [ ]:
data2 = data2[np.arange(len(data2)) % 2 == 0]

In [ ]:
data2=data2.reset_index().drop('index',axis=1)

In [ ]:
data2['Sell Date']=data2['Buy Date'].shift(-1)
data2['Sell Price']=data2['Buy Price'].shift(-1)
data2['avg sell']=data2['avg buy'].shift(-1)

In [ ]:
data2 = data2[np.arange(len(data2)) % 2 == 0]

In [ ]:
data2=data2.reset_index().drop('index',axis=1)

In [ ]:
data2

,Buy Date,Buy Price,avg buy,Sell Date,Sell Price,avg sell
0,2021-04-08 10:30:00+05:30,2731.449951,2729.450012,2021-04-08 15:00:00+05:30,2710.199951,2707.887512
1,2021-04-09 10:30:00+05:30,2743.000000,2733.287415,2021-04-09 15:00:00+05:30,2834.000000,2833.212524
2,2021-04-12 10:30:00+05:30,2878.000000,2860.312500,2021-04-12 15:00:00+05:30,2739.949951,2746.637451


In [ ]:
data['BT Price']=data2['avg sell']
data['ST Price']=data2['avg buy'].shift(-1)

In [ ]:
data['lowest']=data['lowest'].shift(-1)

In [ ]:
final=data.copy()

In [ ]:
final=final.drop(['High','Adj Close','Volume'],axis=1)

In [ ]:
final=final.dropna()

In [ ]:
final

,Date,Open,Low,Close,color,Sash,lowest,BT Price,ST Price
1,2021-04-09,2710.0,2710.0,2832.300049,Green,Bull Sash,2732.350098,2833.212524,2860.3125


In [ ]:
for i in final['Sash']:
  if i!='Bull Sash':
    final.drop(final[final['Sash'] == i].index, inplace = True) 

In [ ]:
final=final.rename(columns={'Low':'Stop Loss','Date':'Candle Date'})

In [ ]:
final=final.drop(['Open','color','Sash','Close'],axis=1)

In [ ]:
final.reset_index(inplace=True)

In [ ]:
final=final.drop('index',axis=1)

In [ ]:
final

,Candle Date,Stop Loss,lowest,BT Price,ST Price
0,2021-04-09,2710.0,2732.350098,2833.212524,2860.3125


In [ ]:
tp=[]
for i in range(0,len(final)):
  if (final['Stop Loss'][i] - ((final['Stop Loss'][i]*0.2)/100)) >= final['lowest'][i]:
    total=(final['Stop Loss'][i] - ((final['Stop Loss'][i]*0.2)/100)) - final['BT Price'][i]
  else:
    total=final['ST Price'][i] - final['BT Price'][i]
  tp.append(total)

final['Total Points'] = tp

In [ ]:
final

,Candle Date,Stop Loss,lowest,BT Price,ST Price,Total Points
0,2021-04-09,2710.0,2732.350098,2833.212524,2860.3125,27.099976


In [ ]:
final['P/L'] = round(10000/final['BT Price'])*final['Total Points']

In [ ]:
a,b= sum(final['P/L']) , len(final) 

In [ ]:
bull_sash_stocks={}
bull_sash_stocks['ssk']=[a,b]
bull_sash_stocks['babu']=[a,b]


In [ ]:
bull_sash_stocks

{'babu': [108.39990234375, 1], 'ssk': [108.39990234375, 1]}

In [ ]:
# final_bull_sash=pd.DataFrame.from_dict(bull_sash_stocks,orient='index',columns=['P/L','counts'],)
# final_bull_sash.reset_index(inplace=True)
# final_bull_sash=final_bull_sash.rename(columns={'index':'Ticker'})
# data3.to_csv('data3.csv')

In [ ]:
def bullsash(stock_name,start_date,end_date,Amount):
  # Read daily data
  data = yf.download(stock_name, 
                      start=start_date, 
                      end=end_date,
                      progress=False)
  # data.dropna(inplace=True)
  # decide candle color
  value=[]
  for i in zip(data['Open'],data['Close']):
    if i[0]>i[1]:
      value.append('Red')
    else:
      value.append('Green')
  data['color']= value
  # decide bull sash
  sash=[]
  for i in range(0,len(data)):
    try:
      if (data['Close'][i] < data['Open'][i+1]) and (data['Open'][i] > data['Open'][i+1]) and (data['Open'][i] < data['Close'][i+1]) and (data['color'][i] == 'Red') and (data['color'][i+1] == 'Green'):
        sash.append('Bull Sash')
      else:
        sash.append('flat')
    except:
      sash.append('flat')
  data['Sash']= sash
  data['Sash']=data['Sash'].shift(1)
  # 15min data of same stock
  data2 = yf.download(stock_name, 
                       start=start_date, 
                       end=end_date,
                       progress=False, interval= "15m")
  #data2.dropna(inplace=True)
  data2.reset_index(inplace=True)
  # sep date time
  data2['Datetime']=data2['Datetime'].astype('str')
  df=data2['Datetime'].str.split(' ',expand=True)
  data2['Date']=df[0]
  data2['Time']=df[1]

  # NSE WAS CLOSE DUE TO GLICTH

  for i in data2['Date']:
    if i=='2021-02-24':
      data2.drop(data2[data2['Date'] == i].index, inplace = True) 

  data2.reset_index(inplace=True)
  data2.drop('index',axis=1,inplace=True)

  data=data.reset_index()

  data['Date']=data['Date'].astype('str')

  for i in data['Date']:
    if i=='2021-02-24':
      data.drop(data[data['Date'] == i].index, inplace = True) 

  data=data.reset_index()
  data.drop('index',axis=1,inplace=True)

  # continue
  data3=data2.copy()
  # remove all other time data
  for i in data2['Time']:
    if i=='10:30:00+05:30' or i=='10:45:00+05:30' or i=='15:00:00+05:30' or i=='15:15:00+05:30':
      pass
    else:
      data2.drop(data2[data2['Time'] == i].index, inplace = True) 
  data2.reset_index(inplace=True)
  data2.drop('index',axis=1,inplace=True)
  #data2
  avg_cost=[]
  for i in range(0,len(data2)-1):
    avg_cost.append(((data2['High'][i] + data2['Low'][i] + data2['High'][i+1] + data2['Low'][i+1])/4))
  avg_cost.append(data2['Open'][i])
  data2['avg buy']=avg_cost
  # get lowest price from 9.15 to 10.30
  for i in data3['Time']:
    if i=='10:30:00+05:30'  or i=='09:15:00+05:30' or i=='09:30:00+05:30' or i=='09:45:00+05:30' or i=='10:00:00+05:30' or i=='10:15:00+05:30':
      pass
    else:
      data3.drop(data3[data3['Time'] == i].index, inplace = True) 
  data3=data3.reset_index()
  low=[]
  for i in list(data['Date']):
    temp=data3[data3['Date'] == i] 
    low.append(min(temp['Low']))
  #add lowest price info to main data
  data['lowest']=low
  # continue # get bt st price
  data2=data2.reset_index().drop('index',axis=1)
  data2=data2.drop(['High','Low','Close','Adj Close','Volume','Date','Time'],axis=1)
  data2=data2.rename(columns={'Open':'Buy Price','Datetime':'Buy Date'})
  data2 = data2[np.arange(len(data2)) % 2 == 0]
  data2=data2.reset_index().drop('index',axis=1)
  data2['Sell Date']=data2['Buy Date'].shift(-1)
  data2['Sell Price']=data2['Buy Price'].shift(-1)
  data2['avg sell']=data2['avg buy'].shift(-1)
  data2 = data2[np.arange(len(data2)) % 2 == 0]
  data2=data2.reset_index().drop('index',axis=1)
  # add bt st price to main data
  data['BT Price']=data2['avg sell']
  data['ST Price']=data2['avg buy'].shift(-1)
  # shift lowest by 1 to get st lowest
  data['lowest']=data['lowest'].shift(-1)
  # create final data
  final=data.copy()
  final=final.drop(['High','Adj Close','Volume'],axis=1)
  final=final.dropna()
  for i in final['Sash']:
    if i!='Bull Sash':
      final.drop(final[final['Sash'] == i].index, inplace = True) 
  final=final.rename(columns={'Low':'Stop Loss','Date':'Candle Date'})
  final=final.drop(['Open','color','Sash','Close'],axis=1)
  final.reset_index(inplace=True)
  final=final.drop('index',axis=1)
  # create total points
  tp=[]
  for i in range(0,len(final)):
    if (final['Stop Loss'][i] - ((final['Stop Loss'][i]*0.2)/100)) >= final['lowest'][i]:
      total=(final['Stop Loss'][i] - ((final['Stop Loss'][i]*0.2)/100)) - final['BT Price'][i]
    else:
      total=final['ST Price'][i] - final['BT Price'][i]
    tp.append(total)

  final['Total Points'] = tp
  final['P/L'] = round(Amount/final['BT Price'])*final['Total Points']
  return sum(final['P/L']) , len(final) 

In [ ]:
bullsash('ALKEM.NS','2021-04-08','2021-04-13',10000)

(0, 0)

In [ ]:
from tqdm import tqdm
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [ ]:
%cd '/content/gdrive/MyDrive/SASH/'

/content/gdrive/MyDrive/SASH


In [ ]:
nse=pd.read_csv('/content/gdrive/MyDrive/SASH/final_bull_sash.csv')

In [ ]:
check_dates=[['2021-04-08','2021-04-13'],['2021-04-09','2021-04-14']]

In [ ]:
for j in check_dates:
  bull_sash_stocks={}
  for i in tqdm(nse['Ticker']):
    try:
      bull_sash_stocks[i]=bullsash(i,j[0],j[1],10000)
    except:
      pass
  final_bull_sash=pd.DataFrame.from_dict(bull_sash_stocks,orient='index',columns=['P/L','counts'],)
  final_bull_sash.reset_index(inplace=True)
  final_bull_sash=final_bull_sash.rename(columns={'index':'Ticker'})
  final_bull_sash.to_csv(j[1] + '.csv')

In [ ]:
# 

In [ ]:
all_stocks=pd.read_excel('/content/gdrive/MyDrive/SASH/Trading_NSE.xlsx')
fando=pd.read_excel('/content/gdrive/MyDrive/SASH/List-of-FO-Stocks-converted.xlsx')

In [ ]:
fno={}
for i in zip(all_stocks['Ticker'],all_stocks['Name'].str.lower()):
  fno[i[1]]=i[0]

final_fo={}
for i in fando['Stock Name'].str.lower():
  for j in fno.items():
    if i==j[0]:
      final_fo[i]=j[1]

fando['Stock Name']=fando['Stock Name'].str.lower()


fntick=[]
for i in fando['Stock Name']:
  if i in final_fo:
    fntick.append(final_fo[i])
  else:
    fntick.append("")

fando['Ticker']=fntick
fando.to_csv('ALL_FNO.csv',index=False)


In [ ]:
fando=pd.read_csv('ALL_FNO.csv')

In [ ]:
fando['Symbol']=fando['Stock Symbol'] + fando['TICKER']

In [ ]:
fando=fando.drop(['Stock Symbol','TICKER'],axis=1)

In [ ]:
fando

,Stock Name,Lot Size,Symbol
0,apollo hospitals enterpirse,500,APOLLOHOSP.NS
1,acc limited,500,ACC.NS
2,amara raja batteries limited.,1000,AMARAJABAT.NS
3,adani enterprises limited,2000,ADANIENT.NS
4,adani port & sez limited,2500,ADANIPORTS.NS
...,...,...,...
131,mindtree limited,800,MINDTREE.NS
132,indiabulls hsg fin limited,3100,IBULHSGFIN.NS
133,vedanta limited,6200,VEDL.NS
134,sun tv network limited,1500,SUNTV.NS


In [ ]:
nta=[]
for i in fando['Symbol']:
  try:
    data = yf.download(i, 
                      start='2021-04-08', 
                      end='2021-04-09',
                      progress=False)
  except:
    pass


1 Failed download:
- RECLIMITED.NS: No data found, symbol may be delisted


In [ ]:
fando.to_csv('ALL_FNO.csv',index=False)